In [36]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import time

  # Replace with your file's ID
#list of articles
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
#list of districts and alloted funds
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
# past public data
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"


inst_data_id = "1dOMubywUqJId2gXHwNWp185L3QmadUnwxyFf0DC9M1s"
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"
master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


In [37]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [38]:
def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [ ]:
articles_df = read_file(article_data_id)
# Find invalid article names
def valid_articles(one):
    invalid_articles = set(one) - set(articles_df["Articles"])

    # Output result
    if invalid_articles:
        print("These articles are not in the master list:")
        print("\n".join(invalid_articles))
    else:
        print("All articles are valid.")


## Code


In [39]:
district_df = read_file(master_data_id)
district_df

Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type
0,Chengalpattu,Agri Power Sprayer (1.2 Bolo),Article,2,11760.0,23520.0,160000.0,NaN,No,District
1,Chengalpattu,Gents Cycle,Article,8,5500.0,44000.0,NaN,NaN,No,District
2,Chengalpattu,Girls Cycle,Article,8,5200.0,41600.0,NaN,NaN,No,District
3,Chengalpattu,Iron Box,Article,8,7000.0,56000.0,NaN,NaN,No,District
4,Chengalpattu,Sewing Machine ORD,Article,6,5200.0,31200.0,NaN,-36320.0,No,District
...,...,...,...,...,...,...,...,...,...,...
77,Thiruppur,Education Aid,Aid,1,50000.0,50000.0,440000.0,NaN,L. Swetha. (B.E),District
78,Thiruppur,Education Aid,Aid,1,15000.0,15000.0,NaN,NaN,M. Abinaya (BSc),District
79,Thiruppur,Education Aid,Aid,1,15000.0,15000.0,NaN,360000.0,M. Madhumitha (BSc),District
80,Tirupathur,Ex gratia for Deceased,Aid,2,25000.0,50000.0,585000.0,NaN,"For 2 families, each Rs.25000",District


In [40]:
district_req = district_df.groupby("REQUESTED ARTICLE")["QUANTITY"].sum()
district_req

REQUESTED ARTICLE
Agri Battery Sprayer                                 38
Agri Power Sprayer (1.2 Bolo)                         3
Aluminium Idli Making Box                             3
Audio Mixer                                           1
Barber Set                                            1
Bosch Electrician Kit 10 Re                           3
Bosch Electrician Kit 13 Re                           2
Bosch Rotary Hammer GBH 220                           1
Business Aid                                          1
Construction Aid                                      1
Desktop Computer                                      2
Diamond Cooker 12 Ltrs                              131
Education Aid                                        15
Epson Printer L3250 (Lite)                            7
Ex gratia for Deceased                                2
Fishing net                                           1
Gents Cycle                                          23
Girls Cycle                   

In [34]:
public_df = read_file(public_master_id)
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,Article Name,Cost Per Unit,Quantity,Total Value,Comments,Beneficiary Type
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,15000,1,15000,Rs.15000 for children,Public
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public
...,...,...,...,...,...,...,...,...,...,...,...,...
98,P 099,R.Dhanalakshmi,781582601881,No,"Pallavaram, Chennai",9840709035,Sewing Machine ORD / Motor,6300,1,6300,No,Public
99,P 100,T.Manjula,556131198246,Yes,"Iduvampalayam, Tirupur",8525852422,Business Aid,5000,1,5000,No,Public
100,P 101,M.Kumar,612081203379,No,"Ozkalur, Kancheepuram",9597058028,Business Aid,10000,1,10000,No,Public
101,P 102,M.Vinayagam,498101932330,Yes,"Ozhalur, Chenglepat",7418225869,Financial Aid,0,1,0,25% of the cost of Handicapped scooter,Public
